In [1]:
import catch22

In [2]:
#assuming full dataset is given 

def pivot(lp1_train,col_name,index_name,columns):
  '''
  used to augment dataset for catch22/TE/sax
  '''
  pivot_datasets = []
  #create a dataframe for every variable
  for i in columns:
    df_pivot = lp1_train.loc[:,[col_name,index_name, i]].pivot(index=index_name, columns=col_name)
    pivot_datasets.append(df_pivot)

  #generate features for each variable
  catch_features = []
  for j in range(len(pivot_datasets)):
    features = generate(lp1_train, pivot_datasets[j], columns[j])
    catch_features.append(features)
    #print(features.shape)

  #pd.concat(catch_features, axis = 1)

  return pivot_datasets, catch_features

In [3]:
def generate(df, df_pivot, name):
  data = []
  
  #find the length of each id
  position_list = list(df.groupby(col_name).count()[index_name])

  #sum length of each id
  sum_post_list = []
  total = 0
  for i in position_list:
    total += i
    sum_post_list.append(total)

  #------- ensure that no nan values are included

  for instance in range(df_pivot.shape[1]):
    try:
      #print(len(df_pivot.iloc[:position_list[instance],instance]))
      catchOut = catch22.catch22_all(df_pivot.iloc[:position_list[instance],instance]) #transpose of data is not in a (1 x N) matrix
      featureNames = catchOut['names']
      featureValues = catchOut['values']
      data.append(featureValues)
    except:
      #print(len(df_pivot.iloc[0:sum_post_list[0],instance]))
      catchOut = catch22.catch22_all(df_pivot.iloc[0:sum_post_list[0],instance]) #transpose of data is not in a (1 x N) matrix
      featureNames = catchOut['names']
      featureValues = catchOut['values']
      data.append(featureValues)

    #print('\n', featureNames, '\n', featureValues)

  for i in range(len(featureNames)):
    featureNames[i] += '_' + name
  #output final dataset
  catch22_dataset_X_train = pd.DataFrame(data, columns=featureNames)

  return catch22_dataset_X_train

# 1. Catch22: Univariate Feature Engineering with Robot Failure Dataset (LP1)

In [4]:
import pandas as pd
import catch22

#load dataset
lp1 = pd.read_csv(r'C:\Users\seren\Documents\RR Lab Documents\lp1_fulldata.csv')
y = lp1['class'].copy()
lp1 = lp1.iloc[:, :4]

lp1.head()

,id,time,class,var1
0,1,1,3,-1
1,1,2,3,0
2,1,3,3,-1
3,1,4,3,-1
4,1,5,3,-1


In [5]:
#define the 3 features
col_name = lp1.columns[0]
index_name = lp1.columns[1]
columns = lp1.columns[3:]

pivot_datasets, catch_features = pivot(lp1,col_name,index_name,columns)

#Catch22 Features
catch_features = pd.concat(catch_features, axis = 1)

In [6]:
print('Extracted Feature set shape: ', catch_features.shape)
catch_features['class'] = y
print('Extracted Feature set shape with class labels: ', catch_features.shape)

Extracted Feature set shape:  (88, 22)
Extracted Feature set shape with class labels:  (88, 23)


In [7]:
catch_features.head()

,DN_HistogramMode_5_var1,DN_HistogramMode_10_var1,CO_f1ecac_var1,CO_FirstMin_ac_var1,CO_HistogramAMI_even_2_5_var1,CO_trev_1_num_var1,MD_hrv_classic_pnn40_var1,SB_BinaryStats_mean_longstretch1_var1,SB_TransitionMatrix_3ac_sumdiagcov_var1,PD_PeriodicityWang_th0_01_var1,...,DN_OutlierInclude_p_001_mdrmd_var1,DN_OutlierInclude_n_001_mdrmd_var1,SP_Summaries_welch_rect_area_5_1_var1,SB_BinaryStats_diff_longstretch0_var1,SB_MotifThree_quantile_hh_var1,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1_var1,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1_var1,SP_Summaries_welch_rect_centroid_var1,FC_LocalSimple_mean3_stderr_var1,class
0,-0.216025,-0.378043,1,1,0.192067,0.000000,0.571429,2.0,0.074830,3,...,-0.066667,0.200000,0.000000e+00,2.0,1.078992,0.0,0.0,2.356194,1.278711,3
1,-0.219718,-0.439435,1,4,0.283070,1.610000,0.714286,3.0,0.047619,0,...,0.066667,0.133333,5.135813e-35,3.0,1.239659,0.0,0.0,1.178097,1.318799,3
2,-0.039977,-0.219874,1,1,0.464883,3.326867,0.785714,2.0,0.078231,3,...,0.466667,-0.333333,0.000000e+00,2.0,1.093375,0.0,0.0,2.356194,0.923233,3
3,-0.113462,-0.283654,1,1,0.303883,-2.728950,0.500000,2.0,0.098639,2,...,-0.466667,-0.600000,0.000000e+00,2.0,0.979971,0.0,0.0,1.963495,0.884333,3
4,-0.134622,0.067311,1,1,0.687092,0.882229,0.714286,3.0,0.057823,0,...,0.200000,0.066667,2.054325e-34,2.0,1.272090,0.0,0.0,2.748894,1.416716,3


# 2. Catch22: Multivariate Feature Engineering with Robot Failure Dataset (LP1)

In [8]:
import pandas as pd
import catch22

#load dataset
lp1 = pd.read_csv(r'C:\Users\seren\Documents\RR Lab Documents\lp1_fulldata.csv')
y = lp1['class'].copy()

lp1.head()

,id,time,class,var1,var2,var3,var4,var5,var6
0,1,1,3,-1,0,57,-5,-3,0
1,1,2,3,0,-3,63,-1,0,0
2,1,3,3,-1,1,51,-4,-1,-1
3,1,4,3,-1,-2,68,-2,-2,0
4,1,5,3,-1,-1,65,-6,1,0


In [26]:
#define the 3 features
col_name = lp1.columns[0]
index_name = lp1.columns[1]
columns = lp1.columns[3:]

pivot_datasets, catch_features = pivot(lp1,col_name,index_name,columns)

#Catch22 Features
catch_features = pd.concat(catch_features, axis = 1)

In [27]:
print('Extracted Feature set shape: ', catch_features.shape)
print('Number of features extracted per sensor: ', catch_features.shape[1]/6)
catch_features['class'] = y[::15].reset_index(drop = True)
print('Extracted Feature set shape with class labels: ', catch_features.shape)

Extracted Feature set shape:  (88, 132)
Number of features extracted per sensor:  22.0
Extracted Feature set shape with class labels:  (88, 133)


In [28]:
catch_features.head()

,DN_HistogramMode_5_var1,DN_HistogramMode_10_var1,CO_f1ecac_var1,CO_FirstMin_ac_var1,CO_HistogramAMI_even_2_5_var1,CO_trev_1_num_var1,MD_hrv_classic_pnn40_var1,SB_BinaryStats_mean_longstretch1_var1,SB_TransitionMatrix_3ac_sumdiagcov_var1,PD_PeriodicityWang_th0_01_var1,...,DN_OutlierInclude_p_001_mdrmd_var6,DN_OutlierInclude_n_001_mdrmd_var6,SP_Summaries_welch_rect_area_5_1_var6,SB_BinaryStats_diff_longstretch0_var6,SB_MotifThree_quantile_hh_var6,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1_var6,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1_var6,SP_Summaries_welch_rect_centroid_var6,FC_LocalSimple_mean3_stderr_var6,class
0,-0.216025,-0.378043,1,1,0.192067,0.000000,0.571429,2.0,0.074830,3,...,0.066667,0.266667,4.622232e-34,2.0,0.000000,0.0,0.0,2.356194,1.259719,3
1,-0.219718,-0.439435,1,4,0.283070,1.610000,0.714286,3.0,0.047619,0,...,0.133333,0.800000,8.217301e-34,2.0,0.754997,0.0,0.0,0.785398,1.121574,3
2,-0.039977,-0.219874,1,1,0.464883,3.326867,0.785714,2.0,0.078231,3,...,-0.066667,0.466667,0.000000e+00,2.0,0.509137,0.0,0.0,2.356194,1.388659,3
3,-0.113462,-0.283654,1,1,0.303883,-2.728950,0.500000,2.0,0.098639,2,...,0.600000,0.066667,1.543027e-34,3.0,1.871160,0.0,0.0,1.178097,1.275250,3
4,-0.134622,0.067311,1,1,0.687092,0.882229,0.714286,3.0,0.057823,0,...,-0.133333,0.866667,2.516548e-33,2.0,0.000000,0.0,0.0,0.785398,0.805076,3


In [24]:
catch_features.to_csv(r'C:\Users\seren\Documents\oct\catch_multi.csv', index = False)

In [25]:
is_NaN = catch_features.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = catch_features[row_has_NaN]

rows_with_NaN

,DN_HistogramMode_5_var1,DN_HistogramMode_10_var1,CO_f1ecac_var1,CO_FirstMin_ac_var1,CO_HistogramAMI_even_2_5_var1,CO_trev_1_num_var1,MD_hrv_classic_pnn40_var1,SB_BinaryStats_mean_longstretch1_var1,SB_TransitionMatrix_3ac_sumdiagcov_var1,PD_PeriodicityWang_th0_01_var1,...,DN_OutlierInclude_p_001_mdrmd_var6,DN_OutlierInclude_n_001_mdrmd_var6,SP_Summaries_welch_rect_area_5_1_var6,SB_BinaryStats_diff_longstretch0_var6,SB_MotifThree_quantile_hh_var6,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1_var6,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1_var6,SP_Summaries_welch_rect_centroid_var6,FC_LocalSimple_mean3_stderr_var6,class
38,0.129099,-0.06455,1,1,0.000000,0.0,0.142857,2.0,0.227891,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
43,0.129099,-0.06455,1,4,0.006418,0.0,0.142857,2.0,0.227891,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
